In [25]:
import cv2
import numpy as np

img = cv2.imread("forms//49553.png")
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
(thresh, img_bin) = cv2.threshold(img_gray, 188, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
img_bin = cv2.bitwise_not(img_bin)

def remote_line(img,t=0):
    (thresh, img_bin) = cv2.threshold(img_gray, 188, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    edges = cv2.Canny(img_gray,100,150,apertureSize = 3)
    if t == 0:
        lines = cv2.HoughLines(edges,1,np.pi/180,120)
        for i in range(len(lines)):
            for rho,theta in lines[i]:
                a = np.cos(theta)
                b = np.sin(theta)
                x0 = a*rho
                y0 = b*rho
                x1 = int(x0 + 1000*(-b))
                y1 = int(y0 + 1000*(a))
                x2 = int(x0 - 1000*(-b))
                y2 = int(y0 - 1000*(a))
            cv2.line(img,(x1,y1),(x2,y2),(255,255,255),2)
        return img
        
cv2.imshow("img2",remote_line(img))
cv2.waitKey(0)        

# # img = remote_line(img)
# img_rl = remote_line(img_gray)
# kernel = np.ones((8,8), np.uint8)
# img_dilation = cv2.erode(img_rl, kernel, iterations=1)
# img_dilation = cv2.dilate(img_dilation, kernel, iterations=1)
# contours, hierarchy = cv2.findContours(image=img_dilation, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_SIMPLE)

# max_x = 0
# max_y = 0
# min_x = 10000
# min_y = 10000
# (x,y,_) = img.shape
# for i in contours:
#     for j in range(len(i)):
#         if i[j][0][0] > max_x:
#             max_x = i[j][0][0]
#         if i[j][0][0] < min_x:
#             min_x = i[j][0][0]
#         if i[j][0][1] > max_y:
#             max_y = i[j][0][1]
#         if i[j][0][1] < min_y:
#             min_y = i[j][0][1]
#     if((max_x-min_x)*(max_y-min_y)/(x*y)>= 0.7):
#         max_x = 0
#         max_y = 0
#         min_x = 10000
#         min_y = 10000
#         continue
#     img = cv2.rectangle(img, (min_x,min_y), (max_x,max_y), [0,255,0], 2)
#     max_x = 0
#     max_y = 0
#     min_x = 10000
#     min_y = 10000
# cv2.imshow("img2",img)
# cv2.waitKey(0)

-1

In [110]:
def lines(edges):
    lines = cv2.HoughLinesP(edges, 1, np.pi / 2, 6, None, 50, 10)
        for line in lines[0]:
            pt1 = (line[0], line[1])
            pt2 = (line[2], line[3])
            cv2.line(edges, pt1, pt2, (0, 0, 255), 2) 
    cv2.imshow("img2",edges)
    cv2.waitKey(0)
        
lines(edges)

IndentationError: unexpected indent (<ipython-input-110-92a4f0f0a07c>, line 3)

In [3]:
import cv2
import numpy as np

large = cv2.imread('forms//49527.png')
rgb = cv2.pyrDown(large)
small = cv2.cvtColor(rgb, cv2.COLOR_BGR2GRAY)

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
grad = cv2.morphologyEx(small, cv2.MORPH_GRADIENT, kernel)

_, bw = cv2.threshold(grad, 0.0, 255.0, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 1))
connected = cv2.morphologyEx(bw, cv2.MORPH_CLOSE, kernel)
# using RETR_EXTERNAL instead of RETR_CCOMP
contours, hierarchy = cv2.findContours(connected.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
#For opencv 3+ comment the previous line and uncomment the following line
#_, contours, hierarchy = cv2.findContours(connected.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

mask = np.zeros(bw.shape, dtype=np.uint8)

for idx in range(len(contours)):
    x, y, w, h = cv2.boundingRect(contours[idx])
    mask[y:y+h, x:x+w] = 0
    cv2.drawContours(mask, contours, idx, (255, 255, 255), -1)
    r = float(cv2.countNonZero(mask[y:y+h, x:x+w])) / (w * h)

    if r > 0.45 and w > 8 and h > 8:
        cv2.rectangle(rgb, (x, y), (x+w-1, y+h-1), (0, 255, 0), 2)

cv2.imshow('rects', rgb)
cv2.waitKey(0)

-1

In [38]:
import cv2
import numpy as np

img = cv2.imread("forms//49527.png")
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
(thresh, img_bin) = cv2.threshold(img_gray, 188, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
img_bin = cv2.bitwise_not(img_bin)

def remote_line(img,t=0):
#     img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    (thresh, img_bin) = cv2.threshold(img_gray, 200, 255, cv2.THRESH_BINARY )
    edges = cv2.Canny(img_gray,30,150,apertureSize = 3)
    if t == 0:
        lines = cv2.HoughLines(edges,1,np.pi/180,180)
        for i in range(len(lines)):
            for rho,theta in lines[i]:
                a = np.cos(theta)
                b = np.sin(theta)
                x0 = a*rho
                y0 = b*rho
                x1 = int(x0 + 1000*(-b))
                y1 = int(y0 + 1000*(a))
                x2 = int(x0 - 1000*(-b))
                y2 = int(y0 - 1000*(a))
            cv2.line(img_bin,(x1,y1),(x2,y2),(255,255,255),3)
        Rres = 20
        Thetares = 1*np.pi/180
        Threshold = 1
        minLineLength = 1
        maxLineGap = 100
        lines = cv2.HoughLinesP(img_bin,Rres,Thetares,Threshold)
        N = lines.shape[0]
        for i in range(N):
            x1 = lines[i][0][0]
            y1 = lines[i][0][1]    
            x2 = lines[i][0][2]
            y2 = lines[i][0][3]    
            cv2.line(img_bin,(x1,y1),(x2,y2),(255,255,255),2)
        return img_bin
        
# img = remote_line(img)
img_rl = remote_line(img_gray)
kernel = np.ones((3,3), np.uint8)
img_dilation = cv2.erode(img_rl, kernel, iterations=1)
img_dilation = cv2.dilate(img_dilation, kernel, iterations=1)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 1))
connected = cv2.morphologyEx(img_dilation, cv2.MORPH_CLOSE, kernel)
contours, hierarchy = cv2.findContours(image=connected, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_SIMPLE)

max_x = 0
max_y = 0
min_x = 10000
min_y = 10000
(x,y,_) = img.shape
for i in contours:
    for j in range(len(i)):
        if i[j][0][0] > max_x:
            max_x = i[j][0][0]
        if i[j][0][0] < min_x:
            min_x = i[j][0][0]
        if i[j][0][1] > max_y:
            max_y = i[j][0][1]
        if i[j][0][1] < min_y:
            min_y = i[j][0][1]
    if((max_x-min_x)*(max_y-min_y)/(x*y)>= 0.7):
        max_x = 0
        max_y = 0
        min_x = 10000
        min_y = 10000
        continue
    img = cv2.rectangle(img, (min_x,min_y), (max_x,max_y), [0,255,0], 2)
    max_x = 0
    max_y = 0
    min_x = 10000
    min_y = 10000
cv2.imshow("img2",img)
cv2.waitKey(0)

-1

In [16]:
import cv2
import numpy as np

rgb = cv2.imread("forms//32.png")
small = cv2.cvtColor(rgb, cv2.COLOR_BGR2GRAY)

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
grad = cv2.morphologyEx(small, cv2.MORPH_GRADIENT, kernel)

_, bw = cv2.threshold(grad, 0.0, 255.0, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 1))
connected = cv2.morphologyEx(bw, cv2.MORPH_CLOSE, kernel)
# using RETR_EXTERNAL instead of RETR_CCOMP
contours, hierarchy = cv2.findContours(connected.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
#For opencv 3+ comment the previous line and uncomment the following line
#_, contours, hierarchy = cv2.findContours(connected.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

mask = np.zeros(bw.shape, dtype=np.uint8)

for idx in range(len(contours)):
    x, y, w, h = cv2.boundingRect(contours[idx])
    mask[y:y+h, x:x+w] = 0
    cv2.drawContours(mask, contours, idx, (255, 255, 255), -1)
    r = float(cv2.countNonZero(mask[y:y+h, x:x+w])) / (w * h)

    if r > 0.15 and w > 3 and h > 1:
        cv2.rectangle(rgb, (x, y), (x+w-1, y+h-1), (0, 255, 0), 2)

cv2.imshow('rects', rgb)
cv2.waitKey(0)

-1